In [2]:
import os
import sys

os.chdir("/teamspace/studios/this_studio/nurse-helper-chatbot")

sys.path.append(os.getcwd())


In [4]:
from llm import llm
from graph import graph
from langchain.chains import GraphCypherQAChain
from langchain.prompts.prompt import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about patients info only.
Convert the user's question based on the schema.

Use the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.

Do not return entire nodes or embedding properties.

Example Cypher Statements:

1.Retrieve All Properties of a Patient by Patient ID
This query retrieves all properties of a patient node using their unique Patient_ID. It outputs the entire set of attributes related to the patient, including personal and medical information, enabling a comprehensive overview of their profile.
```
MATCH (p:patient {{Patient_ID: 154}})
RETURN properties(p) AS PatientProperties; 
```

2. To find patient information and its checkup with which physician attends the checkup

```
MATCH (p:patient {{Patient_ID: 1}})-[:has]->(c:checkup)<-[:attend]-(ph:physician)
RETURN 
    p.Name AS PatientName,
    c.Checkup_ID AS CheckupID, 
    c.Patient_ID AS PatientID, 
    c.Diagnosis AS Diagnosis, 
    c.Diagnosis_Condition AS DiagnosisCondition, 
    c.Treatment AS Treatment, 
    c.Medication AS Medication, 
    c.Dosage AS Dosage, 
    c.`Length_of_Stay (LOS)` AS Length_of_Stay, 
    c.Admission_Date AS AdmissionDate, 
    c.Discharge_Date AS DischargeDate,
    c.Discharge_Condition AS DischargeCondition,
    c.Room_Number AS RoomNumber,
    ph.Physician_Name AS PhysicianName, 
    ph.Specialization AS PhysicianSpecialization, 
    ph.License_Number AS PhysicianLicenseNumber, 
    ph.Physician_Contact AS PhysicianContactNumber;
```

3. To find all Properties of a Patient by Identity Number

```
MATCH (p:patient {{Identity_number: "950314-12-1234"}})
RETURN properties(p) AS PatientProperties; 
```



Remember before display the response, you need to list the details in bulletpoint to make it more readable.

Schema:
{schema}

Question:
{question}
"""

patient_cypher_prompt = PromptTemplate.from_template(CYPHER_GENERATION_TEMPLATE)

patient_cypher_qa = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    return_intermediate_steps=True,
    allow_dangerous_requests=True,
    verbose=True,
    cypher_prompt=patient_cypher_prompt
)

In [9]:
## can u give me the patient info with Identity Number 880519-05-9016?
## can u find a patient information and its checkup with which physician attends the checkup for id 123?
## can u give me patient info for id 523?
query = "can u give me patient info for id 523?"

# Invoke the chain with the complete input data
response = patient_cypher_qa.invoke(query)




> Entering new GraphCypherQAChain chain...


Generated Cypher:

MATCH (p:patient {Patient_ID: 523})
RETURN properties(p) AS PatientProperties; 

Full Context:
[{'PatientProperties': {'Name': 'Tan Li Hao', 'Identity_number': '970715-01-8130', 'Age': 27, 'Allergy': 'Penicillin', 'Patient_Contact': '017-8152498', 'Emergency_number': '014-8432928', 'Gender': 'Male', 'Patient_ID': 523, 'Blood_Type': 'AB-', 'Chronic_condition': 'None'}}]

> Finished chain.


In [6]:
from llm import llm
from graph import graph
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.tools import Tool
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain import hub
from utils import get_session_id
from tools.review import get_review
from langchain_community.callbacks.streamlit import StreamlitCallbackHandler
import streamlit as st 


tools = [

    Tool.from_function(
        name="Patient Information",  
        description="Provide information about patient questions using Cypher and context from GraphCypherQAChain",
        func=hospital_cypher_qa, 
    ),
    
]


def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

agent_prompt = PromptTemplate.from_template("""
You are a nursing expert providing information about nursing care.
Be as helpful as possible and return as much information as possible.
Only answer questions related to checkups, hospitals, or patients.

You must rely solely on the information provided in the context and the tools available to you. 
Do not use any pre-trained knowledge.



TOOLS:
------

You have access to the following tools:

{tools}

To use a tool, please follow this format:
```
Thought: Do I need to use a tool? Yes 
Action: [the action to take, should be one of {tool_names}] 
Action Input: [the input to the action] 
Observation: [the result of the action]
```

You **must** use the tools provided for every relevant question. 
Do not decline to use a tool, as it is essential for generating accurate responses.

When generating responses:
Always follow the required format strictly.
Use the actual Cypher examples from the `GraphCypherQAChain` to construct accurate queries.
Validate the information retrieved from the tools before providing a final answer.
If the information is insufficient, indicate that further clarification is needed.

Use the following format to provide a response:
```
Final Answer: [your response here]
```

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}
""")

agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    )

chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)



def generate_response(user_input):
    """
    Create a handler that calls the Conversational agent
    and returns a response to be rendered in the UI
    """

    # Initialize the Streamlit callback handler
    st_callback = StreamlitCallbackHandler(st.container())

    # Call the agent and capture the response
    response = chat_agent.invoke(
        {"input": user_input},
        {"configurable": {"session_id": get_session_id()}, "callbacks": [st_callback]}
    )

    # Return the final output
    return response['output']

In [8]:
query = "can u give me patient info for id 523?"

# Invoke the chain with the complete input data
response = generate_response(query)

response

2024-12-30 17:49:18.610 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:18.611 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:18.612 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:18.612 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:19.959 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:19.959 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.




> Entering new AgentExecutor chain...


2024-12-30 17:49:20.484 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:20.485 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:20.486 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:20.487 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:20.513 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:20.514 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:20.515 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:20.517 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

```
Thought: Do I need to use a tool? Yes 
Action: Patient Information 
Action Input: {"id": 523} 

> Entering new GraphCypherQAChain chain...


Error in StreamlitCallbackHandler.on_llm_end callback: StreamlitAPIException()


...

Error in StreamlitCallbackHandler.on_tool_end callback: StreamlitAPIException()


{'query': '{"id": 523}', 'result': "The patient's name is Tan Li Hao, he is 27 years old, male, with an identity number of 970715-01-8130. His blood type is AB-, and he has no chronic conditions. He is allergic to penicillin. For contact, you can reach him at 017-8152498, and in case of emergency, you can use the number 014-8432928.", 'intermediate_steps': [{'query': '\nMATCH (p:patient {Patient_ID: 523})\nRETURN properties(p) AS PatientProperties;\n'}, {'context': [{'PatientProperties': {'Name': 'Tan Li Hao', 'Identity_number': '970715-01-8130', 'Age': 27, 'Allergy': 'Penicillin', 'Patient_Contact': '017-8152498', 'Emergency_number': '014-8432928', 'Gender': 'Male', 'Patient_ID': 523, 'Blood_Type': 'AB-', 'Chronic_condition': 'None'}}]}]}

2024-12-30 17:49:27.399 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:27.400 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:27.400 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:27.401 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:27.402 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:27.403 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:27.404 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:49:27.404 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

```
Final Answer: The patient's name is Tan Li Hao. He is a 27-year-old male with an identity number of 970715-01-8130. His blood type is AB-, and he has no chronic conditions. He is allergic to penicillin. For contact, you can reach him at 017-8152498, and in case of emergency, you can use the number 014-8432928.
```

> Finished chain.


"The patient's name is Tan Li Hao. He is a 27-year-old male with an identity number of 970715-01-8130. His blood type is AB-, and he has no chronic conditions. He is allergic to penicillin. For contact, you can reach him at 017-8152498, and in case of emergency, you can use the number 014-8432928.\n```"